In [4]:
import os
import folium
from jinja2 import Template 
import json

print(folium.__version__)

0.7.0


In [5]:
import pandas as pd


old_state_geo = os.path.join('data', 'low_custom.geo.json')#'us-states.json')
country_csv = os.path.join('data', 'output.csv')#'US_Unemployment_Oct2012.csv')

country_data = pd.read_csv(country_csv)

with open(old_state_geo) as f:
    data = json.load(f)
    i=0
    correct=0
    for d in data['features']:
        if   d['properties']['sov_a3'] in country_data['Country_code'].values:
            d['properties']['gap']=str(round(country_data.loc[country_data['Country_code'] == (d['properties']['sov_a3']),'Gap'].values[0]/1000000,2)) + ' billions'
            d['properties']['date']=str(country_data.loc[country_data['Country_code'] == (d['properties']['sov_a3']),'Date'].values[0])
            
        else:
            d['properties']['gap']='n/a'
            d['properties']['date']='n/a'

with open('data/revamped.json', 'w') as outfile:
    json.dump(data, outfile)
state_geo='data/revamped.json'
    

In [6]:
m = folium.Map(
    location=[8, 34],
    zoom_start=3,
    tiles='cartodbpositron'
)



m.choropleth(
    state_geo,
    data=country_data,
    columns=['Country_code', 'Visgap'],
    key_on='feature.properties.sov_a3',
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.2,
    line_color ='black',
    line_weight =1,
    legend_name='ln(Gap) value'
)



g = folium.GeoJson(state_geo, style_function=lambda x: {'weight' : 1, 'opacity' : 0.8, 'fillColor':'#050505', 'fillOpacity': 0, 'color':'#000000'})
g._template = Template("""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = {};

                {{this.get_name()}}.style = function(feature) {return feature.properties.style;};

                {{this.get_name()}}.highlightStyle = function(feature) {
                    return { 
                        weight : 3, 
                        opacity : 0.8};
                    };

                {{this.get_name()}}.onEachFeature = function onEachFeature(feature, layer) {
                    layer.on({
                        mouseover: function(e) {
                            e.target.setStyle({{this.get_name()}}.highlightStyle(e.target.feature));
                            info.update(e.target.feature.properties);},
                        mouseout: function(e) {
                            {{this.get_name()}}.geoJson.resetStyle(e.target);
                            info.update();},
                        click: function(e) {
                            {{this._parent.get_name()}}.fitBounds(e.target.getBounds());}
                        });
                    };

                {{this.get_name()}}.geoJson = L.geoJson(
                    {% if this.embed %}{{this.style_data()}}{% else %}"{{this.data}}"{% endif %},{
                        style : {{this.get_name()}}.style,
                        onEachFeature: {{this.get_name()}}.onEachFeature
                        }).addTo({{this._parent.get_name()}});
                
                var info = L.control({position: 'bottomright'});
                info.onAdd = function ({{this.get_name()}}) {
                    this._div = L.DomUtil.create('div', 'info'); 
                    this.update();
                    return this._div;
                };

                
                info.update = function (props) {
                    this._div.innerHTML =   (props ?
                    '<h1>' + props.name + '</h1>  <h2>' + props.gap + ' USD gap in ' + props.date + '</h2>'
                    : 'Hover over a country');
                };
                



                info.addTo({{this._parent.get_name()}});

               
                {% endmacro %}
""")

m.add_child(g)

popup='This is the largest gap'
folium.CircleMarker(
    location=[27, 29],
    radius=10,
    fill=True,
    color='#FFFF00',
    popup=popup,
    weight=1,
).add_to(m)


m.save(os.path.join('results', 'HackathonFLOC.html'))

m

c:\users\olivier\miniconda3\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
